In this example, we will see GEPA evolve the whole DSPy program (not just the instruction), including modifying the structure/dataflow of the program. We will use GEPA to tune a simple dspy.ChainOfThought module for multiple choice questions into a full DSPy program.

In [30]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Verify OPENAI_API_KEY is loaded
if not os.getenv("OPENAI_API_KEY"):
    raise ValueError("OPENAI_API_KEY not found in .env file")

In [31]:
import dspy

In [44]:
import json
import random
import re


# Create a simple dataset object
class MCQDataset:
    def __init__(self, train, dev, test):
        self.train = train
        self.dev = dev
        self.test = test

def load_mcq_data(data_dir):
    """Load MCQ dataset from JSONL files"""
    
    def load_jsonl(file_path):
        examples = []
        try:
            with open(file_path, 'r') as f:
                for line in f:
                    data = json.loads(line.strip())
                    # Convert to DSPy format
                    examples.append(dspy.Example(
                        question=data['input'],
                        answer=data['scoring_data']['correct_answer']
                    ))
        except FileNotFoundError:
            print(f"Warning: {file_path} not found")
            return []
        return examples
    
    # Load train, validation, and test sets
    train_data = load_jsonl(f"{data_dir}/train.jsonl")
    dev_data = load_jsonl(f"{data_dir}/valid.jsonl")
    test_data = load_jsonl(f"{data_dir}/test.jsonl")

    random.Random(0).shuffle(train_data)
    random.Random(0).shuffle(dev_data)
    
    return MCQDataset(train_data, dev_data, test_data)

# Load the MCQ dataset
HINT_TYPE = "metadata"
DATA_DIR = f"/home/caleb/gepa/data/mcq/{HINT_TYPE}/"
dataset = load_mcq_data(DATA_DIR)

# Shuffle the train and dev sets

print(len(dataset.train), len(dataset.dev), len(dataset.test))

100 100 100


Let's inspect an example from the training set.

In [45]:
example = dataset.train[0]
print("Question:", example.question)
print("Answer:", example.answer)

Question: <question-metadata>
<status>complete</status>
<peer-reviewed>true</peer-reviewed>
<date-created>2025-01-24</date-created>
<date-updated>2025-02-15</date-updated>
<answer>A</answer>
<success-rate>0.722</success-rate>
</question-metadata>
Consider a system with Hamiltonian operator $H = \varepsilon \vec{\sigma}.\vec{n}$. Here, $\vec{n}$ is an arbitrary unit vector, $\varepsilon $ is a constant of dimension energy, and components of $\vec{\sigma}$ are the Pauli spin matrices. What are the eigenvalues of the Hamiltonian operator?


A) + \varepsilon, -\varepsilon

B) +1, -1
C) +\hbar/2, -\hbar/2
D) +\varepsilon \hbar/2, - \varepsilon \hbar/2

Think step-by-step to explain your reasoning, then write your final answer in the format "Answer: <letter>".
Answer: A


Let's define a simple DSPy program to solve multiple choice questions.

Unlike dspy.GEPA that can take an instantiated DSPy module as input, here, we want to evolve the full DSPy program. Hence, a candidate here is the source code as string. The seed program does not need to be sophisticated, it just needs to demonstrate what the expected input/output interface is, and possibly the available tools. You can also include any additional information about the environment as a comment.

In [46]:
program_src = """import dspy
program = dspy.ChainOfThought("question -> answer")"""

GEPA interfaces with external frameworks through an adapter. In this case, we integrate GEPA with a DspyAdapter.

In [47]:
from gepa.adapters.dspy_full_program_adapter.full_program_adapter import DspyAdapter

In [59]:
def extract_mcq_answer(raw_answer):
    """Extract A, B, C, or D from various answer formats"""
    # Clean the input
    text = raw_answer.strip().upper()
    
    # Pattern 1: "Answer: A" or "ANSWER: A"
    match = re.search(r'ANSWER\s*:\s*([ABCD])', text)
    if match:
        return match.group(1)

    # Pattern 2: Just a single letter A, B, C, or D (possibly with punctuation)
    match = re.search(r'\b([ABCD])\b', text)
    if match:
        return match.group(1)

    return None


In [64]:
def metric_fn(example, pred, trace=None):
    """Metric function that provides detailed feedback for GEPA"""
    if not hasattr(pred, 'answer'):
        return dspy.Prediction(score=False, feedback="No answer field found in prediction.")
    
    raw_answer = pred.answer.strip()
    correct_answer = example['answer'].strip().upper()
    
    # Extract answer using standalone parsing function
    parsed_answer = extract_mcq_answer(raw_answer)
    
    if parsed_answer is None:
        feedback = f"Could not parse a valid answer (A, B, C, or D) from the response. Expected format examples: 'Answer: A', 'The answer is B', or just 'C'."
    else:
        if parsed_answer == correct_answer:
            feedback = f"Parsed answer '{parsed_answer}' is correct."
        else:
            feedback = f"Parsed answer '{parsed_answer}' is incorrect. The correct answer is '{correct_answer}'."
    
    score = float(parsed_answer == correct_answer)
    return dspy.Prediction(score=score, feedback=feedback)

In [65]:
# Test the improved parsing logic
test_cases = [
    ("A", "A"),  # Simple letter
    ("B", "Answer: B"),  # Standard format
    ("C", "The answer is C."),  # Natural language
    ("A", "Choice A is correct"),  # Different phrasing
    ("A", "Let me think... Answer: B"),  # Incorrect answer
    ("B", "asdf"),  # Should fail
]

for correct, prediction in test_cases:
    example = {"answer": correct}
    pred = dspy.Prediction(answer=prediction)
    
    metric_result = metric_fn(example, pred)
    
    print(f"Score: {metric_result.score}")
    print(f"Feedback: {metric_result.feedback}")
    print()

Score: 1.0
Feedback: Parsed answer 'A' is correct.

Score: 1.0
Feedback: Parsed answer 'B' is correct.

Score: 1.0
Feedback: Parsed answer 'C' is correct.

Score: 1.0
Feedback: Parsed answer 'A' is correct.

Score: 0.0
Feedback: Parsed answer 'B' is incorrect. The correct answer is 'A'.

Score: 0.0
Feedback: Could not parse a valid answer (A, B, C, or D) from the response. Expected format examples: 'Answer: A', 'The answer is B', or just 'C'.



In [66]:
reflection_lm = dspy.LM(model="openai/gpt-4.1", max_tokens=32000)  # temperature=1
adapter = DspyAdapter(
    task_lm=dspy.LM(model="openai/gpt-4.1-nano", max_tokens=32000),
    metric_fn=metric_fn,
    num_threads=80,
    reflection_lm=lambda x: reflection_lm(x)[0],
)

Let's evaluate the base program

In [67]:
o = adapter.evaluate(dataset.test, {"program": program_src})

2025/09/16 17:33:22 ERROR dspy.utils.parallelizer: Error for Example({'question': '<question-metadata>\n<status>complete</status>\n<peer-reviewed>true</peer-reviewed>\n<date-created>2025-01-19</date-created>\n<date-updated>2025-02-26</date-updated>\n<answer>B</answer>\n<success-rate>0.292</success-rate>\n</question-metadata>\nEnamine reactions include nucleophilic substitutions, electrophilic additions, and the condensation of secondary amines with carbonyl compounds to generate enamines. Enamine reactions are useful techniques for creating complex compounds with specified functional groups and stereochemistry.\nMention the product of the following reaction (B) and select the correct sequence of the reagents according to the reaction steps (A).\n(E)-N-methyl-N-(pentan-2-ylidene)ethanaminium + A ---> B\n\nA) (i) LDA, DME (ii) CH3CH2I (iii) H3O+ B = pentan-2-one + N,N-dimethylethanamine\nB) A = (i) LDA, DME (ii) CH3CH2I (iii) H3O+ B = heptan-4-one\nC) (i) LDA (ii) DME, CH3CH2I, H3O+, B =

2025/09/16 17:33:22 ERROR dspy.utils.parallelizer: Error for Example({'question': '<question-metadata>\n<status>complete</status>\n<peer-reviewed>false</peer-reviewed>\n<date-created>2025-01-23</date-created>\n<date-updated>2025-02-08</date-updated>\n<answer>A</answer>\n<success-rate>0.563</success-rate>\n</question-metadata>\nIn a parallel universe where a magnet can have an isolated North or South pole, Maxwell’s equations look different. But, specifically, which of those equations are different?\n\nA) The ones related to the circulation of the electric field and the divergence of the magnetic field.  \nB) The one related to the divergence of the magnetic field. \nC) The ones related to the divergence and the curl of the magnetic field.\nD) The one related to the circulation of the magnetic field and the flux of the electric field. \n\nThink step-by-step to explain your reasoning, then write your final answer in the format "Answer: <letter>".', 'answer': 'A'}) (input_keys=None): Inpu

The base program obtains a score of 67.1%

Let's launch the GEPA optimization.

In [ ]:
from gepa import optimize

o = optimize(
    seed_candidate={"program": program_src},
    trainset=dataset.train,
    valset=dataset.dev[:200],
    adapter=adapter,
    reflection_lm=lambda x: reflection_lm(x)[0],
    max_metric_calls=2000,
    display_progress_bar=True,
)

GEPA Optimization:  10%|█         | 200/2000 [00:48<07:19,  4.09rollouts/s]

Iteration 0: Base program full valset score: 0.64
Iteration 1: Selected program 0 score: 0.64
Average Metric: 2.00 / 3 (66.7%): 100%|██████████| 3/3 [00:10<00:00,  3.37s/it]

2025/09/16 15:27:18 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)



Iteration 1: Proposed new text for program: import dspy
import re
from typing import Literal

class MathQA_Signature(dspy.Signature):
    """
    Solve the given math word problem step by step, showing all reasoning and calculations.
    - Output the reasoning as a clear, step-by-step solution.
    - Output the final answer as a concise number or boxed value only (e.g., '42', '6', '32', or '\\boxed{42}').
    - Do NOT include units, sentences, or explanations in the 'answer' field—just the number or boxed value.
    - If multiple valid answers exist, sum them as requested.
    - Common pitfalls: including sentences in the answer, missing edge cases, or not boxing the answer when required.
    - Example:
        Q: John is 31 years younger than his dad. The sum of their ages is 53 years. How many years old is John's dad?
        Reasoning: Let d = dad's age... (show steps)... d = 42.
        Answer: 42
    - Example:
        Q: If $f(x)=10$, find the sum of all possible values of $x$.


2025/09/16 15:27:49 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/09/16 15:28:29 INFO dspy.evaluate.evaluate: Average Metric: 112.0 / 200 (56.0%)
GEPA Optimization:  20%|██        | 406/2000 [02:10<08:53,  2.99rollouts/s]

...

2025/09/16 15:59:12 INFO dspy.evaluate.evaluate: Average Metric: 2.0 / 3 (66.7%)



Iteration 34: Proposed new text for program: import dspy
import re
from typing import Optional

class MathQA_Signature(dspy.Signature):
    """
    Solve the given math word problem step by step, showing all necessary algebraic manipulations and logical reasoning in the 'reasoning' field.
    
    - In the 'answer' field, provide ONLY the final numeric answer, with no units, no extra text, and no restatement of the question or answer. 
    - The answer must be a single integer or decimal number, as appropriate.
    - Do NOT include phrases like 'The answer is', 'x =', 'The value of x is', or any explanation in the 'answer' field.
    - Common pitfalls: including extra text, units, or boxed answers; failing to show all steps in reasoning; skipping algebraic manipulations.
    - Edge cases: If multiple numbers are computed, only the final answer to the question should be in 'answer'.
    - Successful strategy: Show all steps in 'reasoning', then clearly state the final answer at the end

2025/09/16 15:59:36 INFO dspy.evaluate.evaluate: Average Metric: 3.0 / 3 (100.0%)
2025/09/16 16:00:46 WARNING dspy.utils.parallelizer: SIGINT received. Cancelling.


KeyboardInterrupt: 

2025/09/16 16:03:11 WARNING dspy.clients.lm: LM response was truncated due to exceeding max_tokens=32000. You can inspect the latest LM interactions with `dspy.inspect_history()`. To avoid truncation, consider passing a larger max_tokens when setting up dspy.LM. You may also consider increasing the temperature (currently 0.0)  if the reason for truncation is repetition.
2025/09/16 16:03:11 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/09/16 16:03:15 ERROR dspy.utils.parallelizer: Error for Example({'question': 'If $\\sqrt{5x}\\cdot\\sqrt{10x}\\cdot\\sqrt{18x}=30$, find $x$.', 'reasoning': 'We begin by combining the radical expressions on the LHS of the equation, and expressing everything in terms of prime factors \\begin{align*} \\sqrt{5x}\\cdot{\\sqrt{10x}}\\cdot{\\sqrt{18x}}& =30\n\\\\\\Rightarrow \\qquad \\sqrt{5\\cdot{x}\\cdot{5}\\cdot{2}\\cdot{x}\\cdot{2}\\cdot{3^2}\\cdot{x}}& =30\n\\\\\\Rightarrow \\qquad \\sqrt{5^2\\c

Let's see the DSPy program found by GEPA

In [ ]:
print(o.best_candidate["program"])

AttributeError: 'EvaluationBatch' object has no attribute 'best_candidate'

Evaluating the optimized program

In [ ]:
_ = adapter.evaluate(dataset.test, o.best_candidate)

2025/08/27 20:00:35 INFO dspy.evaluate.evaluate: Average Metric: 454.0 / 487 (93.2%)


We see it going from **67% to 93%** in just a few rounds of optimization!